In [1]:
import numpy as np

In [2]:
# Sioux Falls network data
# http://www.bgu.ac.il/~bargera/tntp/

#Link travel time = free flow time * ( 1 + B * (flow/capacity)^Power ).
#Link generalized cost = Link travel time + toll_factor * toll + distance_factor * distance

# Traffic Assignment Data structure

class TA_Data(object):
    
    def __init__(self, network_name, \
                 number_of_zones, number_of_nodes, number_of_links, \
                 start_node, end_node, capacity, free_flow_time, \
                 B, power, total_od_flow, travel_demand, od_pairs):
        self.network_name = network_name
        self.number_of_zones = number_of_zones
        self.number_of_nodes = number_of_nodes
        self.number_of_links = number_of_links
        self.start_node = start_node
        self.end_node = end_node
        self.capacity = capacity
        self.free_flow_time = free_flow_time
        self.B = B
        self.power = power
        self.total_od_flow = total_od_flow
        self.travel_demand = travel_demand
        self.od_pairs = od_pairs

In [3]:
def load_ta_network(network_name):
    if network_name == 'Sioux Falls':
        network_data_file = './data/SiouxFalls_net.txt'
        trip_table_file = './data/SiouxFalls_trips.txt' 
    else:
        raise Exception("Invalid network_name; use 'Sioux Falls'.")

    ################
    # Network Data
    ################
    start_node = []
    end_node = []
    capacity = []
    free_flow_time = []
    B = []
    power = []

    with open(network_data_file) as network_data_file_:
        for line in network_data_file_:
            if '<NUMBER OF ZONES>' in line:
                number_of_zones = [int(s) for s in line.split() if s.isdigit()][0]
            elif '<NUMBER OF NODES>' in line:
                number_of_nodes = [int(s) for s in line.split() if s.isdigit()][0]
            elif '<NUMBER OF LINKS>' in line:
                number_of_links = [int(s) for s in line.split() if s.isdigit()][0]
                assert(number_of_links > 0)
            elif '<FIRST THRU NODE>' in line or '<END OF METADATA>' in line or '~' in line or line == '\n':
                continue
            else:
                start_node.append(int(line.split()[0]))
                end_node.append(int(line.split()[1]))
                capacity.append(float(line.split()[2]))
                free_flow_time.append(float(line.split()[4]))
                B.append(float(line.split()[5]))
                power.append(int(line.split()[6]))

    ##############
    # Trip Table
    ##############
    travel_demand = np.zeros((number_of_zones, number_of_zones))
    od_pairs = []

    with open(trip_table_file) as trip_table_file_:
        for line in trip_table_file_:
            if '<NUMBER OF ZONES>' in line:
                number_of_zones_trip = [int(s) for s in line.split() if s.isdigit()][0]
                # Check if number_of_zone is same in both txt files
                assert(number_of_zones_trip == number_of_zones) 
            elif '<TOTAL OD FLOW>' in line:
                total_od_flow = [float(s) for s in line.split() if s.replace('.','',1).isdigit()][0]
                assert(total_od_flow > 0)
            elif '<END OF METADATA>' in line or line == '\n':
                continue
            else:
                if 'Origin' in line:
                    origin = int(line.split()[1])
                elif ';' in line:
                    pairs = line.split(';')
                    for i in range(len(pairs)):
                        if ':' in pairs[i]:
                            pair = pairs[i].split(':')
                            destination = int(pair[0])
                            od_flow = float(pair[1])
                            travel_demand[origin-1, destination-1] = od_flow
                            od_pairs.append((origin, destination))

    ta_data = TA_Data(
    network_name,
    number_of_zones,
    number_of_nodes,
    number_of_links,
    start_node,
    end_node,
    capacity,
    free_flow_time,
    B,
    power,
    total_od_flow,
    travel_demand,
    od_pairs)

    return ta_data


In [4]:
ta_data = load_ta_network('Sioux Fall')

Exception: Invalid network_name; use 'Sioux Falls'.

In [5]:
ta_data = load_ta_network('Sioux Falls')

In [7]:
ta_data.capacity

[25900.20064,
 23403.47319,
 25900.20064,
 4958.180928,
 23403.47319,
 17110.52372,
 23403.47319,
 17110.52372,
 17782.7941,
 4908.82673,
 17782.7941,
 4947.995469,
 10000.0,
 4958.180928,
 4947.995469,
 4898.587646,
 7841.81131,
 23403.47319,
 4898.587646,
 7841.81131,
 5050.193156,
 5045.822583,
 10000.0,
 5050.193156,
 13915.78842,
 13915.78842,
 10000.0,
 13512.00155,
 4854.917717,
 4993.510694,
 4908.82673,
 10000.0,
 4908.82673,
 4876.508287,
 23403.47319,
 4908.82673,
 25900.20064,
 25900.20064,
 5091.256152,
 4876.508287,
 5127.526119,
 4924.790605,
 13512.00155,
 5127.526119,
 14564.75315,
 9599.180565,
 5045.822583,
 4854.917717,
 5229.910063,
 19679.89671,
 4993.510694,
 5229.910063,
 4823.950831,
 23403.47319,
 19679.89671,
 23403.47319,
 14564.75315,
 4823.950831,
 5002.607563,
 23403.47319,
 5002.607563,
 5059.91234,
 5075.697193,
 5059.91234,
 5229.910063,
 4885.357564,
 9599.180565,
 5075.697193,
 5229.910063,
 5000.0,
 4924.790605,
 5000.0,
 5078.508436,
 5091.256152,
